In [1]:
from ctypes import *
dll = WinDLL("CTA-railway.dll")

In [2]:
dll.initSim()

376

In [3]:
dll.runSim()
dll.SimIsFinished()

0

In [4]:
dll.getTotalTravelTime.restype = c_double   
totalTravelTime = dll.getTotalTravelTime()/3600.0
print('%.1f' % totalTravelTime)

231643.6


In [5]:
dll.getTotalDelay.restype = c_double   
totalDelay = dll.getTotalDelay()/3600.0
print('%.1f' % totalDelay)

68741.2


In [ ]:
# reset测试
dll.resetSim()
dll.runSim()
dll.getTotalDelay.restype = c_double   
totalDelay = dll.getTotalDelay()/3600.0
print('%.1f' % totalDelay)

In [ ]:
# 效率测试
import time

time_start=time.time()
for i in range(10):
    dll.resetSim()
    dll.runSim()
    dll.getTotalDelay.restype = c_double   
    totalDelay = dll.getTotalDelay()/3600.0
time_end=time.time()
print('time test (s):\t',(time_end-time_start)/10)

In [6]:
# 分析各个站的残留人数
import numpy as np

dll.getStationWaitingPassengers.argtypes = [c_int, c_int]
dll.getStationWaitingPassengers.restype = c_int
count = 0
lefts = []
for station in range(252):
    buffer = [station]
    buffer.append(dll.getStationWaitingPassengers(station, 0))
    buffer.append(dll.getStationWaitingPassengers(station, 1))
    buffer.append(buffer[1]+buffer[2])
    lefts.append(buffer)
    count += dll.getStationWaitingPassengers(station, 0)
    count += dll.getStationWaitingPassengers(station, 1)
print(count, " passengers left in the stations after the Simulation")

left = np.array(lefts)
top_left = left[np.lexsort(-left.T)]
for i in range(252):
#     dir = 0
#     if (top_left[i][2] > 0):
#         dir = 1
#     if (top_left[i][3] > 0):
#         print("Station ", '%d'%top_left[i][0], "\tProblem direction: \t", dir)
    print("Station ID: ", '%.0f'%top_left[i][0], "\tDirection0 Left passengers: ", '%d'%(top_left[i][1]),
          "\tDirection1 Left passengers: ", '%d'%(top_left[i][2]), "\tLeft passengers: ", '%d'%(top_left[i][3]))
    

# left_stations = []
# for i in range(252):
#     if top_left[i][3] > 0:
#         left_stations.append(top_left[i][0])
#         print('%d'%top_left[i][0])

0  passengers left in the stations after the Simulation
Station ID:  251 	Direction0 Left passengers:  0 	Direction1 Left passengers:  0 	Left passengers:  0
Station ID:  250 	Direction0 Left passengers:  0 	Direction1 Left passengers:  0 	Left passengers:  0
Station ID:  249 	Direction0 Left passengers:  0 	Direction1 Left passengers:  0 	Left passengers:  0
Station ID:  248 	Direction0 Left passengers:  0 	Direction1 Left passengers:  0 	Left passengers:  0
Station ID:  247 	Direction0 Left passengers:  0 	Direction1 Left passengers:  0 	Left passengers:  0
Station ID:  246 	Direction0 Left passengers:  0 	Direction1 Left passengers:  0 	Left passengers:  0
Station ID:  245 	Direction0 Left passengers:  0 	Direction1 Left passengers:  0 	Left passengers:  0
Station ID:  244 	Direction0 Left passengers:  0 	Direction1 Left passengers:  0 	Left passengers:  0
Station ID:  243 	Direction0 Left passengers:  0 	Direction1 Left passengers:  0 	Left passengers:  0
Station ID:  242 	Directio

In [7]:
# 分析各个站的delay
import numpy as np

dll.getStationDelay.argtypes = [c_int, c_int]
dll.getStationDelay.restype = c_double
delays = []
for station in range(252):
    buffer = [station]
    buffer.append(dll.getStationDelay(station, 0))
    buffer.append(dll.getStationDelay(station, 1))
    buffer.append(buffer[1]+buffer[2])
    delays.append(buffer)

delay = np.array(delays)
top_delay = delay[np.lexsort(-delay.T)]
for i in range(252):
    print("Station ID: ", '%.0f'%top_delay[i][0], "\tDelay (h): ", '%.1f'%(top_delay[i][3]/3600))

Station ID:  192 	Delay (h):  8485.3
Station ID:  223 	Delay (h):  6432.2
Station ID:  231 	Delay (h):  6002.9
Station ID:  4 	Delay (h):  3447.0
Station ID:  92 	Delay (h):  3139.9
Station ID:  78 	Delay (h):  2135.9
Station ID:  85 	Delay (h):  1887.9
Station ID:  15 	Delay (h):  1840.2
Station ID:  67 	Delay (h):  1042.4
Station ID:  25 	Delay (h):  869.6
Station ID:  178 	Delay (h):  851.4
Station ID:  230 	Delay (h):  845.5
Station ID:  71 	Delay (h):  808.4
Station ID:  24 	Delay (h):  732.6
Station ID:  52 	Delay (h):  688.5
Station ID:  23 	Delay (h):  685.3
Station ID:  194 	Delay (h):  596.6
Station ID:  115 	Delay (h):  588.9
Station ID:  167 	Delay (h):  584.8
Station ID:  28 	Delay (h):  559.2
Station ID:  185 	Delay (h):  545.2
Station ID:  35 	Delay (h):  542.4
Station ID:  235 	Delay (h):  509.3
Station ID:  89 	Delay (h):  492.2
Station ID:  22 	Delay (h):  474.7
Station ID:  179 	Delay (h):  454.1
Station ID:  170 	Delay (h):  418.2
Station ID:  96 	Delay (h):  358.1


In [10]:
# 分析各个站的人均delay
import numpy as np

dll.getStationPass.argtypes = [c_int, c_int]
dll.getStationPass.restype = c_int
avg_delays = []
for station in range(252):
    buffer = [station]
    if dll.getStationPass(station, 0) != 0:
        buffer.append(dll.getStationDelay(station, 0)/float(dll.getStationPass(station, 0)))
    else:
        buffer.append(0)
    if dll.getStationPass(station, 1) != 0:
        buffer.append(dll.getStationDelay(station, 1)/float(dll.getStationPass(station, 1)))
    else:
        buffer.append(0)
    buffer.append(max(buffer[1],buffer[2]))
#     buffer.append(dll.getStationPass(station, 0))
#     buffer.append(dll.getStationPass(station, 1))
#     buffer.append(buffer[1]+buffer[2])
    avg_delays.append(buffer)

avg_delay = np.array(avg_delays)
top_avg_delay = avg_delay[np.lexsort(-avg_delay.T)]
for i in range(252):
#     print("Station ID:  ", '%.0f'%top_avg_delay[i][0], "\tNumPass0: ", '%d\t'%(top_avg_delay[i][1]), 
#           "\tNumPass1: ", '%d'%(top_avg_delay[i][2]))
    print("Station ID:  ", '%d'%top_avg_delay[i][0], "\tAvg_delay0: ", '%.1f\t'%(top_avg_delay[i][1]),
          "\tAvg_delay1: ", '%.1f'%(top_avg_delay[i][2]))

Station ID:   192 	Avg_delay0:  3183.0	 	Avg_delay1:  0
Station ID:   179 	Avg_delay0:  2511.1	 	Avg_delay1:  0
Station ID:   167 	Avg_delay0:  2218.4	 	Avg_delay1:  0
Station ID:   231 	Avg_delay0:  2105.1	 	Avg_delay1:  0
Station ID:   221 	Avg_delay0:  2023.3	 	Avg_delay1:  0
Station ID:   141 	Avg_delay0:  2014.3	 	Avg_delay1:  0
Station ID:   223 	Avg_delay0:  1435.2	 	Avg_delay1:  0
Station ID:   170 	Avg_delay0:  1290.1	 	Avg_delay1:  0
Station ID:   157 	Avg_delay0:  1232.5	 	Avg_delay1:  0
Station ID:   66 	Avg_delay0:  658.1	 	Avg_delay1:  418
Station ID:   44 	Avg_delay0:  0.0	 	Avg_delay1:  582
Station ID:   56 	Avg_delay0:  572.8	 	Avg_delay1:  528
Station ID:   17 	Avg_delay0:  0.0	 	Avg_delay1:  544
Station ID:   153 	Avg_delay0:  0.0	 	Avg_delay1:  454
Station ID:   246 	Avg_delay0:  386.8	 	Avg_delay1:  0
Station ID:   186 	Avg_delay0:  0.0	 	Avg_delay1:  373
Station ID:   160 	Avg_delay0:  0.0	 	Avg_delay1:  366
Station ID:   203 	Avg_delay0:  0.0	 	Avg_delay1:  363
S

In [ ]:
import csv
stations_file = 'data/check_stations.csv'
stations = []
lineCount = 0
problem_set = set()
with open(stations_file, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        lineCount += 1
        if lineCount == 1:
            continue
        stations.append(row)
for station in left_stations:
    for row in stations:
        if station == int(row[-2]):
            problem_set.add(row[4])
print(problem_set)